In [ ]:
using Polyhedra
using DrakeVisualizer, CoordinateTransformations
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

In [ ]:
using Plots; gr()

In [ ]:
vis = Visualizer()[:hopper]
setgeometry!(vis, HyperRectangle(Vec(-0.1, -0.1, 0), Vec(0.2, 0.2, 0.2)))
setgeometry!(vis[:foot], HyperRectangle(Vec(-0.05, -0.05, 0), Vec(0.1, 0.1, 0.1)))

In [ ]:
include("examples/articulated_complementarity.jl")

In [ ]:

env = Complementarity.Environment(
    [
        Complementarity.Obstacle(
            SimpleHRepresentation{2, Float64}([0 1], [0]),
            HalfSpace{2, Float64}([0, 1], 0)
        ),
        Complementarity.Obstacle(
            SimpleHRepresentation{2, Float64}([-1 0], [-0.2]),
            HalfSpace{2, Float64}([-1, 0], -0.2)
        )
    ],
    [
        SimpleHRepresentation{2, Float64}(
            [0 -1;
             1 0],
            [0, 0.2])
    ]
)

q0 = [-0.5, 1.25, 1.0]
v0 = [0, 0, 0.0]
kp = 1.25
kd = 0.01 * kp
controller = (q, v) -> kp * (1 - q[3]) - kd * v[3]
N = 20

In [ ]:
results = Complementarity.simulate(q0, v0, controller, env, 40);

In [ ]:
for r in results
    settransform!(vis, Translation(r.q[1], 0, r.q[2]))
    settransform!(vis[:foot], Translation(0, 0, -r.q[3]))
    sleep(0.1)
end

In [ ]:
plot([r.q[2] for r in results])